In [ ]:
%pip install --upgrade huggingface_hub
%pip install -U transformers

In [ ]:
%pip install accelerate

In [ ]:
from huggingface_hub import login
from huggingface_hub import hf_hub_download
from google.colab import userdata

cache_folder = '/content/drive/MyDrive/code'

login(userdata.get('HF_TOKEN'))
hf_hub_download(repo_id="google/gemma-2b", filename="config.json", cache_dir=cache_folder)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

#model_path = "/content/drive/MyDrive/code/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2"
model_path = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_path, cache_dir=cache_folder)
model = AutoModelForCausalLM.from_pretrained(model_path , device_map="auto", cache_dir=cache_folder)


In [ ]:
# move cached download to my own space. Then no need to repeat the download.
# it saves time but may lag on versions
# don't change the structure. the definition of model structure is scattered in files.
%mv /root/.cache/huggingface/hub/models--google--flan-t5-base/ /content/drive/MyDrive/code/
%ls /content/drive/MyDrive/code/


In [40]:
def get_gemma_text( str_promp ):
  #input_text = "translate English to German: How old are you?"
  print(str_promp)
  input_text = str_promp
  input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
  outputs = model.generate(
      **input_ids,
      max_new_tokens=40,
      num_beams=5,
      no_repeat_ngram_size=2,
      num_return_sequences=5,
      early_stopping=True
  )
  str_ret = ''
  for dt1 in outputs:
    str_ret += tokenizer.decode(dt1)
    str_ret += '</br></br>'
  return str_ret

In [41]:
# code section for web-app (Flask)
#   part 1: reveal public url

from google.colab import output

output.serve_kernel_port_as_window(8084)

<IPython.core.display.Javascript object>

In [ ]:
#   part 2: web-app page

from flask import Flask, request

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
  str_input = str_t5 = ''
  if (request.method == "POST"):
    str_input = request.form.get("get_prompt")
    str_t5 = get_gemma_text( str_input )

  str_page = """
    <html>
      <body>
        <form action='/' method='post'>
          Put Prompt: <input name='get_prompt' value='{}'> <button type='submit'>Submit</button><br>
          <hr>
          Response: <br>
          {}
        </form>
      </body>
    </html>
  """
  str_page = str_page.format(str_input, str_t5)
  return str_page

app.run(port=8084)

In [ ]:
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))